In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS poc_dataengg.monitoring;

CREATE OR REPLACE TABLE poc_dataengg.monitoring.data_quality_log (
    validation_time TIMESTAMP,
    job_name STRING,
    trade_date DATE,
    rule_name STRING,
    status BOOLEAN COMMENT 'TRUE if all passed, FALSE if any failed',
    detail STRING COMMENT 'Extra info like datatype or row count',
    success_count BIGINT COMMENT 'Number of records that passed this rule',
    failure_count BIGINT COMMENT 'Number of records that failed this rule',
    bad_record STRING COMMENT 'Sample JSON of failing records (optional)'

)
USING DELTA
PARTITIONED BY (trade_date)
;

In [0]:
%sql
Select Count(*) from poc_dataengg.monitoring.data_quality_log

count(1)
15


In [0]:
STORAGE_ACCOUNT = "cxdlbbronze"
CONTAINER = "landingzone"
Folder = "BTC-USD-Partitioned"

lakehouse_path = f"abfss://{CONTAINER}@{STORAGE_ACCOUNT}.dfs.core.windows.net/{Folder}"
checkpoint_path = lakehouse_path + "_checkpoint/last_trade.json"

client_id = dbutils.secrets.get(scope="landingzone-secret", key="client-id")
client_secret = dbutils.secrets.get(scope="landingzone-secret", key="client-secret")
tenant_id = dbutils.secrets.get(scope="landingzone-secret", key="tenant-id")

account_host = f"{STORAGE_ACCOUNT}.dfs.core.windows.net"
spark.conf.set(f"fs.azure.account.auth.type.{account_host}", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{account_host}",
               "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{account_host}", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{account_host}", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{account_host}",
               f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
from pyspark.sql.types import (
    StructType, StructField,
    LongType, DecimalType, StringType,
    TimestampType, DateType
)

expected_schema = StructType([
    StructField("trade_id", LongType(), False),                   # must always be present
    StructField("trade_time", TimestampType(), False),            # parsed from API "time"
    StructField("trade_date", DateType(), False),                 # derived from trade_time
    StructField("price", DecimalType(18, 2), False),              # cast from string
    StructField("size", DecimalType(18, 8), False),               # trade size with high precision
    StructField("side", StringType(), False),                     # "buy" / "sell"
    StructField("ingest_ts", TimestampType(), False)              # ingestion timestamp
])

dq_log_schema = StructType([
    StructField("validation_time", TimestampType(), False),
    StructField("job_name", StringType(), False),
    StructField("trade_date", DateType(), False),
    StructField("rule_name", StringType(), False),
    StructField("status", BooleanType(), False),
    StructField("detail", StringType(), True),
    StructField("success_count", LongType(), True),
    StructField("failure_count", LongType(), True),
    StructField("bad_record", StringType(), True)
])


In [0]:
df_incremental = spark.read.format("delta").load(lakehouse_path).filter("trade_date='2025-09-21'")

In [0]:
df_incremental.count()

1098

In [0]:
from pyspark.sql.functions import col, to_date, to_json, struct
from datetime import datetime

def validate_and_log(df, expected_schema, job_name="btc_usd_pipeline"):
    dq_logs = []
    total_count = df.count()

    # Current trade_date
    trade_date_str = df.selectExpr("max(trade_date)").first()[0]; trade_date_val = datetime.strptime(trade_date_str, '%Y-%m-%d').date()

    # 1. Row Count Rule
    dq_logs.append((datetime.utcnow(), job_name, trade_date_val,
                    "RowCount",
                    total_count > 0,
                    f"{total_count} rows",
                    total_count if total_count > 0 else 0,
                    0 if total_count > 0 else total_count,
                    None))

    # 2. Null Checks
    for field in expected_schema.fields:
        null_df = df.filter(col(field.name).isNull())
        fail_count = null_df.count()
        success_count = total_count - fail_count

        bad_payload = None
        if fail_count > 0:
            bad_json = null_df.withColumn("record", to_json(struct([col(c) for c in df.columns]))) \
                              .select("record").limit(5)  # sample 5
            bad_list = [r.record for r in bad_json.collect()]
            bad_payload = "; ".join(bad_list)

        dq_logs.append((datetime.utcnow(), job_name, trade_date_val,
                        f"NullCheck_{field.name}",
                        fail_count == 0,
                        f"{fail_count} nulls",
                        success_count,
                        fail_count,
                        bad_payload))

    # 3. Schema Checks
    actual_fields = {f.name: f.dataType for f in df.schema.fields}
    for field in expected_schema.fields:
        actual_type = actual_fields.get(field.name)
        dq_logs.append((datetime.utcnow(), job_name, trade_date_val,
                        f"SchemaCheck_{field.name}",
                        actual_type == field.dataType,
                        str(actual_type),
                        total_count if actual_type == field.dataType else 0,
                        0 if actual_type == field.dataType else total_count,
                        None))

    # Convert to DF
    #dq_df = spark.createDataFrame(
    #    dq_logs,
    #    ["validation_time", "job_name", "trade_date", "rule_name",
    #     "status", "detail", "success_count", "failure_count", "bad_record"]
    #).withColumn("trade_date", to_date(col("trade_date")))
    
    dq_df = spark.createDataFrame(dq_logs, schema=dq_log_schema)


    # Append to table
    dq_df.write.format("delta").mode("append").saveAsTable("monitoring.data_quality_log")

    return dq_df

# Run validation and log results
dq_log_df = validate_and_log(df_incremental, expected_schema, job_name="btc_usd_pipeline")

# Show results inline
dq_log_df.show(truncate=False)


/root/.ipykernel/1308/command-6230574895729533-178129781:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  dq_logs.append((datetime.utcnow(), job_name, trade_date_val,
/root/.ipykernel/1308/command-6230574895729533-178129781:33: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  dq_logs.append((datetime.utcnow(), job_name, trade_date_val,
/root/.ipykernel/1308/command-6230574895729533-178129781:45: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  dq_logs.append((datetime.utcnow(), job_name, trade_date_val,


+--------------------------+----------------+----------+----------------------+------+-----------------+-------------+-------------+----------+
|validation_time           |job_name        |trade_date|rule_name             |status|detail           |success_count|failure_count|bad_record|
+--------------------------+----------------+----------+----------------------+------+-----------------+-------------+-------------+----------+
|2025-09-21 13:26:55.087454|btc_usd_pipeline|2025-09-21|RowCount              |true  |1098 rows        |1098         |0            |NULL      |
|2025-09-21 13:26:55.470611|btc_usd_pipeline|2025-09-21|NullCheck_trade_id    |true  |0 nulls          |1098         |0            |NULL      |
|2025-09-21 13:26:55.774582|btc_usd_pipeline|2025-09-21|NullCheck_trade_time  |true  |0 nulls          |1098         |0            |NULL      |
|2025-09-21 13:26:56.134044|btc_usd_pipeline|2025-09-21|NullCheck_trade_date  |true  |0 nulls          |1098         |0            |NULL

In [0]:
%sql
SELECT trade_date, rule_name, SUM(failure_count) AS total_failures
FROM monitoring.data_quality_log
WHERE trade_date = current_date()
GROUP BY trade_date, rule_name;



trade_date,rule_name,total_failures
2025-09-21,SchemaCheck_trade_id,0
2025-09-21,SchemaCheck_side,0
2025-09-21,SchemaCheck_price,0
2025-09-21,NullCheck_trade_id,0
2025-09-21,SchemaCheck_trade_date,1098
2025-09-21,NullCheck_ingest_ts,0
2025-09-21,NullCheck_side,0
2025-09-21,SchemaCheck_ingest_ts,1098
2025-09-21,RowCount,0
2025-09-21,NullCheck_trade_time,0


In [0]:
%sql
SELECT trade_date, 
       rule_name,
       SUM(success_count) AS passed,
       SUM(failure_count) AS failed,
       ROUND(SUM(success_count)*100.0 / (SUM(success_count)+SUM(failure_count)), 2) AS pass_percent
FROM monitoring.data_quality_log
GROUP BY trade_date, rule_name;



trade_date,rule_name,passed,failed,pass_percent
2025-09-21,SchemaCheck_trade_id,1098,0,100.00
2025-09-21,SchemaCheck_side,1098,0,100.00
2025-09-21,SchemaCheck_price,1098,0,100.00
2025-09-21,NullCheck_trade_id,1098,0,100.00
2025-09-21,SchemaCheck_trade_date,0,1098,0.00
2025-09-21,NullCheck_ingest_ts,1098,0,100.00
2025-09-21,NullCheck_side,1098,0,100.00
2025-09-21,SchemaCheck_ingest_ts,0,1098,0.00
2025-09-21,RowCount,1098,0,100.00
2025-09-21,NullCheck_trade_time,1098,0,100.00


In [0]:
%sql

SELECT *
FROM monitoring.data_quality_log
where rule_name in ('SchemaCheck_trade_date','SchemaCheck_ingest_ts')

/*
For rules like SchemaCheck_trade_date and SchemaCheck_ingest_ts, the check is comparing expected type vs actual type.
If they don’t match, you’re marking every row as “failed” (failure_count = total_count).
But since this isn’t a row-level failure (it’s a schema mismatch), there’s no “bad record” subset to log — hence bad_record = NULL.
*/



validation_time,job_name,trade_date,rule_name,status,detail,success_count,failure_count,bad_record
2025-09-21T12:57:13.372391Z,btc_usd_pipeline,2025-09-21,SchemaCheck_trade_date,false,StringType(),0,1098,null
2025-09-21T12:57:13.372425Z,btc_usd_pipeline,2025-09-21,SchemaCheck_ingest_ts,false,StringType(),0,1098,null
